In [1]:
import numpy as np
from collections import Counter
import pandas as pd

In [2]:
# Node class to represent a single decision node or leaf in the tree
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

# Decision Tree class for creating and training a single decision tree
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):  # Fit the tree to the training data
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):  # Recursive function to grow the tree
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):  # Find the best feature and threshold to split
        best_gain = -1
        split_idx, split_threshold = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):  # Compute information gain
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
        return parent_entropy - child_entropy

    def _split(self, X_column, split_thresh):  # Split data based on a threshold
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):  # Calculate entropy of a label distribution
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _most_common_label(self, y):  # Find the most common label in a dataset
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):  # Predict labels for given input data
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):  # Traverse the tree for prediction
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [3]:
# Random Forest class for creating and training multiple decision trees
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):  # Fit the forest by training multiple trees
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features,
            )
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):  # Generate bootstrap samples
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):  # Get the most common label from predictions
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):  # Predict using all trees and majority voting
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        return np.array([self._most_common_label(pred) for pred in tree_preds])

In [4]:
# Load and preprocess the dataset
data = pd.read_csv("CLI.csv")
data['Percent Change'] = pd.to_numeric(data['Percent Change'].str.replace('%', ''), errors='coerce')
data['Volume'] = pd.to_numeric(data['Volume'].str.replace(',', ''), errors='coerce')
data = data.dropna()
features = ['Open', 'High', 'Low', 'Close']
target = 'Percent Change'
X = data[features].values
y = (data[target] > 0).astype(int).values

In [5]:
# Manually split the data into training and test sets
np.random.seed(1234)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
split_idx = int(0.8 * len(indices))
train_indices, test_indices = indices[:split_idx], indices[split_idx:]
X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

In [6]:
# Define accuracy metric
def accuracy(y_true, y_pred):  # Calculate accuracy of predictions
    return np.sum(y_true == y_pred) / len(y_true)

In [7]:
# Train the random forest and evaluate
clf = RandomForest(n_trees=20)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [8]:
acc = accuracy(y_test, predictions)
print("Accuracy:", acc)

Accuracy: 0.75


In [9]:
# Predict the next movement (yes for increase, no for decrease)
last_row = X_test[-1].reshape(1, -1)  # Take the last test data row
next_movement = clf.predict(last_row)[0]
print("yes" if next_movement == 1 else "no")

no
